In [90]:
#ライブラリの自動リロード
%load_ext autoreload
%autoreload 2

from src.GGUFBot import GGUFBot
from src.HFDataset import HFDataset
from src.SimpleQuestionGenerator import SimpleQuestionGenerator
from src.AnswerGenerator import AnswerGenerator
import pandas as pd
import random
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:

#df=pd.read_csv('data/original_q.csv') # indexはなし
#original_question_list=df["question"].values.tolist()
#original_question_list


ds_name="hatakeyama-llm-team/WikiBookJa"
ds_name="allenai/peS2o"
ds_name="kanhatakeyama/TanukiCorpus"
ds_name="hatakeyama-llm-team/WikiBookJa"
ds=HFDataset(ds_name,streaming=True)

#ds_name="suolyer/pile_stackexchange"
#ds=load_dataset(ds_name,split="train",streaming=False).shuffle()

#ds_name="hatakeyama-llm-team/japanese2010"
#ds_name="hpprc/wikipedia-20240101"
#ds_name="sudy-super/JetCopper-10B"
# %%

In [75]:
n_layers=400
max_new_tokens=4000
model_path="/home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf"
bot=GGUFBot(model_path,max_new_tokens=max_new_tokens,n_ctx=max_new_tokens,n_gpu_layers=n_layers)

llama_model_loader: additional 3 GGUFs metadata loaded.
llama_model_loader: loaded meta data with 29 key-value pairs and 563 tensors from /home/hatakeyama/python/ChatServer/model/Mixtral-8x22B-Instruct-v0.1.Q5_K_M-00001-of-00004.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = models--mistralai--Mixtral-8x22B-Inst...
llama_model_loader: - kv   2:                          llama.block_count u32              = 56
llama_model_loader: - kv   3:                       llama.context_length u32              = 65536
llama_model_loader: - kv   4:                     llama.embedding_length u32              = 6144
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 16384
llama_model_loader: - 

loading model...


ggml_cuda_init: GGML_CUDA_FORCE_MMQ:   yes
ggml_cuda_init: CUDA_USE_TENSOR_CORES: no
ggml_cuda_init: found 8 CUDA devices:
  Device 0: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 1: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 2: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 3: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 4: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 5: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 6: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
  Device 7: NVIDIA A100-SXM4-80GB, compute capability 8.0, VMM: yes
llm_load_tensors: ggml ctx size =    2.51 MiB
llm_load_tensors: offloading 56 repeating layers to GPU
llm_load_tensors: offloading non-repeating layers to GPU
llm_load_tensors: offloaded 57/57 layers to GPU
llm_load_tensors:        CPU buffer size =   132.00 MiB
llm_load_tensors:      CUDA0 buffer size = 12238.73 MiB
llm_load_tenso

In [76]:

q_gen=SimpleQuestionGenerator()
a_gen=AnswerGenerator(bot)

In [102]:

save_path="data/"+ds_name.replace("/","_")+".jsonl"
while True:
    try:
        record={}
        record["text"]=next(ds)
        record["database"]=ds_name
        record["inst_question"]=q_gen(record["text"])
        record["question"]=bot.ask(record["inst_question"])
        a_gen(record)

        with open(save_path,"a") as f:
            f.write(json.dumps(record,ensure_ascii=False)+"\n")
    except Exception as e:
        print("error",e)


Llama.generate: prefix-match hit

llama_print_timings:        load time =    1997.54 ms
llama_print_timings:      sample time =      30.46 ms /   125 runs   (    0.24 ms per token,  4104.01 tokens per second)
llama_print_timings: prompt eval time =    2131.30 ms /   345 tokens (    6.18 ms per token,   161.87 tokens per second)
llama_print_timings:        eval time =    6333.63 ms /   124 runs   (   51.08 ms per token,    19.58 tokens per second)
llama_print_timings:       total time =    8753.09 ms /   469 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =    1997.54 ms
llama_print_timings:      sample time =     109.66 ms /   486 runs   (    0.23 ms per token,  4431.92 tokens per second)
llama_print_timings: prompt eval time =    1529.56 ms /   142 tokens (   10.77 ms per token,    92.84 tokens per second)
llama_print_timings:        eval time =   27959.55 ms /   485 runs   (   57.65 ms per token,    17.35 tokens per second)
llama_print_timings:       to

{'text': '法学＞民事法＞コンメンタール＞コンメンタール法の適用に関する通則法\n(婚姻の効力)\n第25条\n婚姻の効力は、夫婦の本国法が同一であるときはその法により、その法がない場合において夫婦の常居所地法が同一であるときはその法により、そのいずれの法もないときは夫婦に最も密接な関係がある地の法による。\n----\n{{前後\n|法の適用に関する通則法\n|第3章 準拠法に関する通則\n第5節親族\n|法の適用に関する通則法第24条(婚姻の成立及び方式)\n|法の適用に関する通則法第26条(夫婦財産制)\n25',
 'inst_question': '以下の文章をもとに、日本語で新しい指示を1つ作成してください。出力には､問題/質問/指示文のみを出力すること｡\n法学＞民事法＞コンメンタール＞コンメンタール法の適用に関する通則法\n(婚姻の効力)\n第25条\n婚姻の効力は、夫婦の本国法が同一であるときはその法により、その法がない場合において夫婦の常居所地法が同一であるときはその法により、そのいずれの法もないときは夫婦に最も密接な関係がある地の法による。\n----\n{{前後\n|法の適用に関する通則法\n|第3章 準拠法に関する通則\n第5節親族\n|法の適用に関する通則法第24条(婚姻の成立及び方式)\n|法の適用に関する通則法第26条(夫婦財産制)\n25',
 'question': ' 問題: 以下の国際結婚の場合、夫婦に最も密接な関係がある地の法により、婚姻の効力はどのように定められますか。\n① 夫婦の本国法が同一である場合\n② 夫婦の常居所地法が同一である場合\n③ いずれの法もない場合',
 'inst_answer_0': '日本語でstep by stepで回答してください:  問題: 以下の国際結婚の場合、夫婦に最も密接な関係がある地の法により、婚姻の効力はどのように定められますか。\n① 夫婦の本国法が同一である場合\n② 夫婦の常居所地法が同一である場合\n③ いずれの法もない場合',
 'answer_0': ' 問題に対する解答は以下の通りです:\n\n1. 夫婦の本国法が同一である場合: 夫婦が同じ国籍を持っている場合、その国の法律が適用されます。このため、婚姻に関する規定は、夫婦の本国の法により決まります。\n